In [1]:
import numpy as np

In [2]:
DEFAULT_THRESH = 10**-12

In [3]:
def f_1(x):
    return np.exp(-(x**2)/2)

def i_simpson(a,b,n,fofx):
    h = (b-a)/n
    i_simp = fofx(a)/6 + fofx(b)/6
    for i in range(1,n):
        i_simp += fofx(a+i*h)/3
    for i in range(1,n+1):
        i_simp += 2*fofx(a+(i-0.5)*h)/3
    return h*i_simp

def norm_cdf(x, n):
    if x > 0:
        return 0.5 + (1 / np.sqrt(2 * np.pi)) * i_simpson(0,x,n,f_1)
    elif x < 0:
        return 0.5 - (1 / np.sqrt(2 * np.pi)) * i_simpson(x,0,n,f_1)
    else:
        return 0.5

def norm_cdf_thresh(x, thresh):
    n_0, n = 4, 8
    i_old, i_new = norm_cdf(x, n_0), norm_cdf(x, n)
    print("using n = %s, estimated integral = %s" % (n_0, i_old))
    print("using n = %s, estimated integral = %s" % (n, i_new))
    while(np.abs(i_new - i_old) > thresh):
        i_old = i_new
        n = 2 * n
        i_new = norm_cdf(x, n)
        print("using n = %s, estimated integral = %s" % (n, i_new))
    return i_new

def norm_cdf_def_thresh(x):
    return norm_cdf_thresh(x, DEFAULT_THRESH)

def norm_cdf_approx(x):
    z = np.abs(x)
    y = 1 / (1 + 0.2316419 * z)
    a1, a2, a3, a4, a5 = 0.319381530, -0.356563782, 1.781477937, -1.821255978, 1.330274429
    m = 1 - np.exp((-x**2)/2)*(a1*y + a2*y**2 + a3*y**3 + a4*y**4 + a5*y**5) / np.sqrt(2*np.pi)
    if x > 0:
        return m
    else:
        return 1-m

In [4]:
def bs_price(T,isCall,S,K,vol,r,q,n_cdf):
    d1 = (np.log(S / K) + (r - q - (vol ** 2)/2) * T) / (vol * np.sqrt(T))
    d2 = d1 - (vol * np.sqrt(T))

    if isCall:
        #call
        return S * np.exp(-q * T) * n_cdf(d1) - K * np.exp(-r * T) * n_cdf(d2)
    else:
        #put
        return K * np.exp(-r * T) * n_cdf(-d2) - S * np.exp(-q * T) * n_cdf(-d1)

In [5]:
def bs_delta(T,isCall,S,K,vol,r,q,n_cdf):
    d1 = (np.log(S / K) + (r - q - (vol ** 2)/2) * T) / (vol * np.sqrt(T))
    if isCall:
        return n_cdf(d1)
    else:
        return -n_cdf(-d1)

In [13]:
from scipy.stats import norm

def bs_delta(T,isCall,S,K,vol,r,q,n_cdf):
    d1 = (np.log(S / K) + (r - q - (vol ** 2)/2) * T) / (vol * np.sqrt(T))
    if isCall:
        return n_cdf(d1)
    else:
        return -n_cdf(-d1)

T = 0.5
S, K = 100, 100
vol = 0.3
r = 0.05
q = 0
isCall = False

put_delta = bs_delta(T,isCall,S,K,vol,r,q,norm.cdf)
print(put_delta)

-0.495298528968


In [7]:
d1 = (np.log(S / K) + (r - q - (vol ** 2)/2) * T) / (vol * np.sqrt(T))
d1

0.011785113019775802

In [9]:
from scipy.stats import norm

In [11]:
norm.cdf(-d1)

0.49529852896770193

In [14]:
put_delta * 1000 * 100

-49529.852896770186

In [16]:
T = 0.5 - 1/252
S, K = 102, 100
vol = 0.3
r = 0.05
q = 0
isCall = False

price_put_new = bs_price(T,isCall,S,K,vol,r,q,norm_cdf_def_thresh)

using n = 4, estimated integral = 0.542140261068
using n = 8, estimated integral = 0.542140261048
using n = 16, estimated integral = 0.542140261047
using n = 32, estimated integral = 0.542140261047
using n = 4, estimated integral = 0.458004851916
using n = 8, estimated integral = 0.458004851935
using n = 16, estimated integral = 0.458004851937
using n = 32, estimated integral = 0.458004851937


In [18]:
print(price_put)
print(price_put_new)

6.97915142647
6.16947434028


In [20]:
(price_put_new - price_put) * 1000

-809.6770861940854

In [21]:
(102-100)*495.2985

990.597

In [22]:
(price_put_new - price_put) * 1000 + (102-100)*495.2985

180.91991380591458